In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle
%matplotlib inline
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import accuracy_score, confusion_matrix,roc_curve,roc_auc_score
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')
from sklearn.tree import DecisionTreeClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import accuracy_score, confusion_matrix,roc_curve,roc_auc_score
from sklearn.metrics import classification_report

In [ ]:
data=pd.read_csv('https://raw.githubusercontent.com/dsrscientist/Data-Science-ML-Capstone-Projects/master/baseball.csv')
data.head()

In [ ]:
data.shape

In [ ]:
data.info()

No imputation or encoding required

In [ ]:
plt.figure(figsize=(20,15),facecolor='red')
plotno=1
for column in data:
    if plotno<=18:
        ax=plt.subplot(5,4,plotno)
        sns.distplot(data[column])
        plt.xlabel(column,fontsize=20)
    plotno+=1
plt.tight_layout()

In [ ]:
data.columns

In [ ]:
sns.scatterplot(x='W',y='R',data=data)

In [ ]:
sns.scatterplot(x='W',y='HR',data=data)

In [ ]:
sns.scatterplot(x='W',y='SV',data=data)

In [ ]:
x=data.drop(columns=['W'])
y=data['W']

In [ ]:
plt.figure(figsize=(20,15),facecolor='blue')
plotnumber=1
for column in x:
    if plotnumber<=18:
        ax=plt.subplot(5,4,plotnumber)
        plt.scatter(x[column],y)
        plt.xlabel(column,fontsize=20)
        plt.ylabel('Wins',fontsize=20)
    plotnumber+=1
plt.tight_layout()

In [ ]:
data_corr=data.corr().abs()
plt.figure(figsize=(15,11))
sns.heatmap(data_corr,annot=True,annot_kws={'size':10})
plt.show

In [ ]:
sns.pairplot(data)
plt.show()

In [ ]:
data.corr()

In [ ]:
data.corr()['W'].sort_values()

Win has good bonds with HR,2B, 3B and SB.
Win has great bond with ERA, RA, ER, R, SHO, BB, SV.
Others have a weak bond.

In [ ]:
data.describe()

Based on heat map, we can see ERA, ER, RA are correlated. we can remove ER and RA as ERA has slightly better bond with outcome W. As well as CG as it has a weak bond with W

In [ ]:
data.skew()

Skewness threshold considered as +/-0.6. R, ERA, SHO, E, SV have skewness

In [ ]:
sns.distplot(data['R'])

In [ ]:
data_features=data.drop('W',axis=1)
plt.figure(figsize=(20,20),facecolor='red')
plotno=1
for column in data_features:
    if plotno<=18:
        ax=plt.subplot(5,5,plotno)
        sns.boxplot(data=data_features[column])
        plt.xlabel(column,fontsize=12)
    plotno+=1
plt.show()

There are outliers present in some features, but the dataset is too small to remove outliers.

In [ ]:
data=data.drop(columns=['ER','RA'],axis=1)

In [ ]:
x=data.drop(columns=['W'])
y=data['W']

In [ ]:
from sklearn.feature_selection import SelectKBest,f_classif

In [ ]:
best_features=SelectKBest(score_func=f_classif,k=14)

fit=best_features.fit(x,y)

data_scores=pd.DataFrame(fit.scores_)

data_columns=pd.DataFrame(x.columns)

feature_scores=pd.concat([data_columns,data_scores],axis=1)

feature_scores.columns=['Feature_Name','Score']

print(feature_scores.nlargest(12,'Score'))

In [ ]:
df=data[['R','SB','SV','E','ERA','AB','SO','SHO','BB','HR','2B','3B','W']]

In [ ]:
x=df.drop(columns=['W'])
y=df['W']

In [ ]:
plt.figure(figsize=(25,25))
plotno=1
for column in x:
    if plotno<=15:
        ax=plt.subplot(4,4,plotno)
        sns.stripplot(y,x[column])
        plt.xlabel(column,fontsize=20)
        plt.ylabel('Wins',fontsize=20)
    plotno+=1
plt.show()

In [ ]:
scalar=StandardScaler()
x_scalar=scalar.fit_transform(x)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x_scalar,y,test_size=0.05,random_state=455)

In [ ]:
regression=LinearRegression()
regression.fit(x_train,y_train)

In [ ]:
regression.score(x_train,y_train)

In [ ]:
regression.score(x_test,y_test)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import Lasso,Ridge,RidgeCV,LassoCV

In [ ]:
y_pred=regression.predict(x_test)

mean_absolute_error(y_test,y_pred)

In [ ]:
mean_squared_error(y_test,y_pred)

In [ ]:
np.sqrt(mean_squared_error(y_test,y_pred))

In [ ]:
print('Score :',regression.predict(scalar.transform([[724,104,56,88,3.73,5575,973,8,383,139,300,42]])))

In [ ]:
df

In [ ]:
lasscv=LassoCV(alphas=None,max_iter=200,normalize=True)
lasscv.fit(x_train,y_train)

In [ ]:
alpha=lasscv.alpha_
alpha

In [ ]:
lasso_reg=Lasso(alpha)
lasso_reg.fit(x_train,y_train)

In [ ]:
lasso_reg.score(x_test,y_test)

In [ ]:
ridgecv=RidgeCV(alphas=np.arange(0.002,0.1,0.02),normalize=True)
ridgecv.fit(x_train,y_train)

In [ ]:
ridgecv.alpha_

In [ ]:
ridge_model=Ridge(alpha=ridgecv.alpha_)
ridge_model.fit(x_train,y_train)

In [ ]:
ridge_model.score(x_test,y_test)

Lasso is giving highest accuracy, so we will move with that

In [ ]:
pickle.dump(lasscv,open('Baseball prediction','wb'))

In [ ]:
cross_val_score(lasscv,x_scalar,y,cv=5)

In [ ]:
cross_val_score(LassoCV(),x_scalar,y,cv=5).mean()